In [1]:
from kss import KSS
from data_utils import Iris
from sklearn import metrics
from sklearn.model_selection import train_test_split


In [2]:
data = Iris()
features, labels = data.get_data()

In [3]:
print(features)
print('---------')
print(labels)

     sepal_lenght  sepal width  petal_length  petal_width
0             5.8          2.8           5.1          2.4
1             6.7          3.0           5.2          2.3
2             7.4          2.8           6.1          1.9
3             5.9          3.2           4.8          1.8
4             5.2          4.1           1.5          0.1
..            ...          ...           ...          ...
145           6.3          2.7           4.9          1.8
146           7.2          3.2           6.0          1.8
147           5.6          2.7           4.2          1.3
148           4.7          3.2           1.3          0.2
149           5.0          3.6           1.4          0.2

[150 rows x 4 columns]
---------
0       Iris-virginica
1       Iris-virginica
2       Iris-virginica
3      Iris-versicolor
4          Iris-setosa
            ...       
145     Iris-virginica
146     Iris-virginica
147    Iris-versicolor
148        Iris-setosa
149        Iris-setosa
Name: class, Leng

In [4]:
X_train,X_test,y_train,y_test = train_test_split(features.to_numpy(),labels.to_numpy(),test_size=0.2,random_state=4)

In [5]:
k_range = range(1,20)

for k in k_range:
  kss = KSS(k)
  kss.fit(X_train,y_train,'SEP')
  y_pred = kss.predict(X_test)

  print(k)

  print(metrics.classification_report(y_test, y_pred))

1
                 precision    recall  f1-score   support

    Iris-setosa       1.00      1.00      1.00         9
Iris-versicolor       0.93      1.00      0.96        13
 Iris-virginica       1.00      0.88      0.93         8

       accuracy                           0.97        30
      macro avg       0.98      0.96      0.97        30
   weighted avg       0.97      0.97      0.97        30

2
                 precision    recall  f1-score   support

    Iris-setosa       1.00      1.00      1.00         9
Iris-versicolor       0.93      1.00      0.96        13
 Iris-virginica       1.00      0.88      0.93         8

       accuracy                           0.97        30
      macro avg       0.98      0.96      0.97        30
   weighted avg       0.97      0.97      0.97        30

3
                 precision    recall  f1-score   support

    Iris-setosa       1.00      1.00      1.00         9
Iris-versicolor       0.93      1.00      0.96        13
 Iris-virginica   